In [1]:
import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#https://drive.google.com/drive/folders/1Cq8lxEXY5TCCyI8r5YrNiVAuUc1KlRqe?usp=sharing

from google.colab import drive
drive.mount('/content/drive')

data_root='/content/drive/My Drive/data_set_flowers/'

Mounted at /content/drive


In [3]:
#separa as imagens em treino, teste e validação.
TRAINING_DATA_DIR = str(data_root+'flowers_train')#recebe as flores de treino
TEST_DATA_DIR = str(data_root+'flowers_test')#recebe as flores de teste
IMAGE_SHAPE = (200,200)#o tamanho das imagens são 200x200
datagen_kwargs = dict(rescale=1./255, validation_split=.20)#faço a reescala das imagens em 255, e validação de 20%, ai os dados de:
valid_datagen = ImageDataGenerator(**datagen_kwargs)#validação
train_datagen = ImageDataGenerator(**datagen_kwargs)#treino
test_datagen = ImageDataGenerator(**datagen_kwargs)#teste

In [4]:
valid_generator = valid_datagen.flow_from_directory(#pego a partir do diretório do drive
    TRAINING_DATA_DIR,#passo o caminho da pasta
    subset="validation",#passo o conjunto de validação(20% atribuido anteriormente)
    shuffle=True,#embaralha
    target_size=IMAGE_SHAPE,#tamanho da imagem
    classes = ['daisy', 'dandelion', 'rose']#as tres possíveis classes
)

test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=IMAGE_SHAPE,
    classes = ['daisy', 'dandelion', 'rose']
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE,
    classes = ['daisy', 'dandelion', 'rose']
)

Found 405 images belonging to 3 classes.
Found 574 images belonging to 3 classes.
Found 1626 images belonging to 3 classes.


In [5]:
modelo = keras.Sequential([keras.layers.Flatten(input_shape=(200, 200, 3)),#cada imagem 200x200 com padrão RGB
keras.layers.Dense(256, activation=tensorflow.nn.relu),
keras.layers.Dense(128, activation=tensorflow.nn.relu),
keras.layers.Dense(train_generator.num_classes, activation='softmax')#saida
])

#compila o modelo 
modelo.compile(
optimizer='Adam',
loss='categorical_crossentropy',
metrics=['acc'])

In [6]:
#passos, quantos passos por epoca
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
#validações por época
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = modelo.fit(
train_generator,#recebe os dados de train_generator
epochs=10,#10 epocas
verbose=1,#imprime
steps_per_epoch=steps_per_epoch,#espaços por época
validation_data=valid_generator,#dados de validação que ele vai receber
validation_steps=val_steps_per_epoch).history #validação por época

Epoch 1/10
51/51 [==============================] - 343s 7s/step - loss: 14.8276 - acc: 0.4323 - val_loss: 2.6395 - val_acc: 0.5580
Epoch 2/10
51/51 [==============================] - 18s 342ms/step - loss: 2.5317 - acc: 0.5117 - val_loss: 3.1216 - val_acc: 0.5012
Epoch 3/10
51/51 [==============================] - 18s 347ms/step - loss: 2.7167 - acc: 0.5025 - val_loss: 1.2858 - val_acc: 0.6025
Epoch 4/10
51/51 [==============================] - 18s 343ms/step - loss: 1.3839 - acc: 0.5935 - val_loss: 2.0801 - val_acc: 0.4370
Epoch 5/10
51/51 [==============================] - 18s 345ms/step - loss: 1.2519 - acc: 0.6070 - val_loss: 1.6957 - val_acc: 0.5457
Epoch 6/10
51/51 [==============================] - 18s 350ms/step - loss: 1.1039 - acc: 0.6513 - val_loss: 1.5334 - val_acc: 0.5259
Epoch 7/10
51/51 [==============================] - 18s 348ms/step - loss: 0.7318 - acc: 0.7140 - val_loss: 1.6867 - val_acc: 0.5432
Epoch 8/10
51/51 [==============================] - 18s 344ms/step - l

In [7]:
#salva o modelo
SAVED_MODEL = "/content/drive/My Drive/data_set_flowers/saved_models"
modelo.save(SAVED_MODEL)
model = keras.models.load_model(SAVED_MODEL)

INFO:tensorflow:Assets written to: /content/drive/My Drive/data_set_flowers/saved_models/assets


In [8]:
val_image_batch, val_label_batch = next(iter(test_generator))#separo novamente os batch, recebendo os test_generator
true_label_ids = np.argmax(val_label_batch, axis=-1)#valores dos labels verdadeiros
print("Validation batch shape:", val_image_batch.shape)#imprime o shape de validação

Validation batch shape: (32, 200, 200, 3)


In [9]:
model.predict(val_image_batch)#faz o predict do val_image_batch

array([[6.5224546e-01, 3.3990496e-01, 7.8495573e-03],
       [5.8927196e-01, 1.9925936e-01, 2.1146865e-01],
       [2.9268894e-01, 6.4380413e-01, 6.3506939e-02],
       [1.2969038e-01, 1.9643307e-02, 8.5066634e-01],
       [2.4734399e-01, 4.6304733e-01, 2.8960869e-01],
       [9.9507755e-01, 4.8944387e-03, 2.8022650e-05],
       [2.1209891e-01, 7.8221136e-01, 5.6897211e-03],
       [4.4066179e-01, 5.4081696e-01, 1.8521182e-02],
       [9.2145878e-01, 7.5647302e-02, 2.8939305e-03],
       [7.0518410e-01, 2.7458498e-01, 2.0230846e-02],
       [5.7060969e-01, 1.7650302e-01, 2.5288728e-01],
       [4.2078400e-01, 5.1797843e-01, 6.1237633e-02],
       [2.3039397e-02, 1.6812226e-02, 9.6014839e-01],
       [4.8526555e-01, 5.1427996e-01, 4.5441845e-04],
       [6.3483006e-01, 3.5702470e-01, 8.1452597e-03],
       [9.5902465e-02, 8.0561763e-01, 9.8479897e-02],
       [2.0894091e-01, 7.4278659e-01, 4.8272520e-02],
       [1.2196152e-04, 4.4298307e-05, 9.9983370e-01],
       [3.6971964e-04, 3.828

In [10]:
#aqui carrego o modelo e peço para avaliar
#passando os dados das imagens e as classes
perda_teste, acuracia_teste=model.evaluate(val_image_batch, val_label_batch)
print('Perda do teste', perda_teste)#imprimo as perdas
print('Acurácia do teste', acuracia_teste)#imprimo as acurácias

1/1 [==============================] - 0s 172ms/step - loss: 0.9951 - acc: 0.5938
Perda do teste 0.9950731992721558
Acurácia do teste 0.59375
